### Author: Nicholas A. Ondo
### License: MIT

---
# Scraping BoxOfficeMojo

I will be scraping BoxOfficeMojo using the following logic:

1. Scrape the pages containing the movies per year, iterating over years and then the pages per year, to create a list of links to the movies that year.
2. Iterate over the list of movies, scraping out the basic information I want (e.g. ).
3. Do basic sanity checks that the data has been successfully scraped from the website.

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

def get_soup_from(url):
    sauce = requests.get("http://www.boxofficemojo.com" + url.strip())
    soup = BeautifulSoup(sauce.text, "lxml")
    
    return soup

---
# Part I: Scraping the Yearly List of Movies

Useful notes:

1. So BoxOfficeMojo (henceforth BOM) has a standardized URL for accessing yearly data, namely:

```
http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&yr=1980
```

which is easily controlled via the ```page=``` and  ```yr=``` flags.  These can be iterated over easily in Python.

2. The database ranges from 1980 (for domestic) and 1990 (for international) grosses.  There's a choice on how far back to use, but for convenience sake, I will grab it all now and decide later.

3. It's easiest just to look for regex patterns ```/movies/?id=``` since this is the pattern for (although it turns out we'll need to ignore the first entry).

In [2]:
url = "http://www.boxofficemojo.com/yearly/chart/?page=5&view=releasedate&yr=1999"
response = requests.get(url)

response.status_code

200

## Scraping the list of movies from a single given page.

In [3]:
soup = BeautifulSoup(response.text, "lxml")

print(len(soup.find_all("a", href=True)))

# NOTE: I chose page 5 for demonstration purposes, there aren't supposed to be any movies here.

55


In [4]:
movie_pattern = re.compile(r"/movies/\?id=")

movie_urls = []
for tag in soup.find_all('a', href=True):
    match = movie_pattern.search(tag["href"])

    if match is not None:
        movie_urls.append(tag["href"])
        
movie_urls

['/movies/?id=hoteltransylvania3.htm']

Notice that without killing the zeroeth entry, the first movie is from 2018, and it's because BOM has this practice of putting the current top grossing film in theaters now at the top of the page.  So we can outright ignore the first entry on every single page.

Looking it over, it seems like the others are totally fine, so I will wait until I scrape more to see if it is a problem on other pages.

## Looping through a given year to scrape all of the movies that year

In [5]:
def scrape_movie_urls(year, page):
    url = f"http://www.boxofficemojo.com/yearly/chart/?page={page}&yr={year}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"CONNECTION FAILED: year={year} at page={page}")
        return None
    
    soup = BeautifulSoup(response.text, "lxml")
    movie_pattern = re.compile(r"/movies/\?id=")
    
    movie_urls = []
    for tag in soup.find_all('a', href=True):
        match = movie_pattern.search(tag["href"])

        if match is not None:
            movie_urls += [tag["href"] + "\n"]

    return movie_urls[1:]


# scrape_movie_urls("1980", "1")

In [6]:
years = [year for year in range(1980,2019)]
pages = [page for page in range(1,5)]

def scrape_all_movie_urls():
    # This is only 60 pages or so, I do not believe I will need to worry about page limiting.
    with open("movie_urls.txt", "w") as f:
        for year in years:
            for page in pages:
                try:
                    urls = scrape_movie_urls(year, page)
                except:
                    urls = [f"FAILED: year={year}, page={page}.  Try this again.\n"]

                f.writelines(urls)

def load_movie_urls():
    try:
        with open("movie_urls.txt", "r") as f:
            urls = f.readlines()
    except:
        urls = scrape_all_movies_urls()
        
    return urls
        
# movie_urls = load_movie_urls()
# movie_urls[:10]

__N.B.__ Note that there are no fails, we have grabbed all of the website URL's!

In [7]:
def scrape_top_table(soup, identifier):                               
    for table_tag in soup.find_all("table", bgcolor=True):             # <table> loop
        for td_tag in table_tag.find_all("td"):                        # <td> loop
            if identifier in td_tag.text:                              # check query condition
                return (td_tag.text
                        .replace(identifier, "")
                        .replace(":", "").strip())


def scrape_player_table(soup, player):
    player_names = []
    for some_divtag in soup.find_all("div", class_="mp_box_content"): # <div> loop
        for tr_tag in some_divtag.find_all("tr"):                     # <tr> loop
            if tr_tag is not None and player in tr_tag.text.strip():  # check query condition
                for a_tag in tr_tag.find_all("a", href=True):         # <a> loop
                    if "*" not in a_tag.text:                         # don't grab minor roles
                        player_names.append(a_tag.text.strip())       # Add the value, including name
                break
    player_names = player_names[1:]
    
    if player_names != []:
        return ",".join(player_names)
    else:
        return "N/A"



In [8]:
# Useful checking values
url1 = "/movies/?id=starwars5.htm"       # 1980
url2 = "/movies/?id=austinpowers2.htm"   # 1999
url3 = "/movies/?id=toystory2.htm"       # 1999
url4 = "/movies/?id=indianajones4.htm"   # 2008
url5 = "/movies/?id=aquietplace.htm"     # 2018

In [9]:
# Write a function that then pulls all of these properties from a soup

def scrape_all_movdata(soup):
    mov_attribs = ["Domestic Total Gross", "Distributor", "Genre",
                   "MPAA Rating", "Release Date", "Runtime", "Production Budget"]
    players = ["Director", "Producer", "Actor"]
    
    vals = {}
    vals["Name"] = soup.find("title").text[:-25]
    
    for id_text in mov_attribs:
        vals[id_text] = [scrape_top_table(soup, id_text)]

    for player in players:
        vals[player] = scrape_player_table(soup, player)

    return pd.DataFrame(vals)

# soup = get_soup_from(url4)
# df = scrape_all_movdata(soup)

# df

In [10]:
def scrape_and_make_df(urls, dt=1.0):
    movie_dfs = []
    for url in urls:
        # Grab the data at each URL
        a_soup = get_soup_from(url)
        movie_dfs.append(scrape_all_movdata(a_soup))

        # Wait for irregular intervals
        time.sleep(np.random.uniform(high=dt))

    return pd.concat(movie_dfs, ignore_index=True)


In [11]:
def scrape_movieyearly_df(year):
    pages = [page for page in range(1,5)]
    urls = []
    for page in pages:
        urls += scrape_movie_urls(year, page)
    
    return scrape_and_make_df([url.strip() for url in urls])


In [12]:
for year in years:
    temp_df = scrape_movieyearly_df(year)
    temp_df.to_csv(f"movies_{year}.csv")
